#GNN

In [ ]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.4.0+cu121


In [ ]:
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb
!pip install fsspec==2024.3.1

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 17.8 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.

In [ ]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

data = dataset[0]
print(data)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:02<00:00, 38.44it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1640.96it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 690.53it/s]

Saving...



Done!
/usr/local/lib/python3.10/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


In [ ]:
print('The graph has {} features'.format(data.num_features))

The graph has 128 features


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [ ]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
data = dataset[0]

data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _repr(self.p

Device: cpu


In [ ]:
for s in split_idx:
  print(s, len(split_idx[s]))

train 90941
valid 29799
test 48603


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim) for _ in range(num_layers - 2)] +
            [GCNConv(in_channels=hidden_dim, out_channels=output_dim)]
        )
        self.bns = torch.nn.ModuleList(
            [torch.nn.BatchNorm1d(num_features=hidden_dim) for _ in range(num_layers - 1)]
        )
        self.softmax = torch.nn.Softmax()
        self.dropout = dropout
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for conv, bn in zip(self.convs[:-1], self.bns):
            x = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x = F.dropout(x, p=self.dropout)
            x = x
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)
        return out

In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].reshape(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [ ]:
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
args

{'device': 'cpu',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [ ]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [ ]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

/usr/local/lib/python3.10/dist-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch: 01, Loss: -0.0243, Train: 11.00%, Valid: 22.97% Test: 21.56%
Epoch: 02, Loss: -0.2669, Train: 36.22%, Valid: 36.05% Test: 33.64%
Epoch: 03, Loss: -0.3813, Train: 28.08%, Valid: 21.08% Test: 24.13%
Epoch: 04, Loss: -0.4332, Train: 29.18%, Valid: 16.59% Test: 14.87%
Epoch: 05, Loss: -0.4598, Train: 32.83%, Valid: 18.37% Test: 14.83%
Epoch: 06, Loss: -0.4745, Train: 41.34%, Valid: 36.28% Test: 32.46%
Epoch: 07, Loss: -0.4873, Train: 45.36%, Valid: 42.56% Test: 39.60%
Epoch: 08, Loss: -0.4965, Train: 46.50%, Valid: 44.81% Test: 43.51%
Epoch: 09, Loss: -0.5086, Train: 46.63%, Valid: 45.27% Test: 44.57%
Epoch: 10, Loss: -0.5230, Train: 47.26%, Valid: 46.56% Test: 45.52%
Epoch: 11, Loss: -0.5366, Train: 47.49%, Valid: 47.27% Test: 46.54%
Epoch: 12, Loss: -0.5411, Train: 48.14%, Valid: 50.69% Test: 51.23%
Epoch: 13, Loss: -0.5440, Train: 48.93%, Valid: 53.35% Test: 55.35%
Epoch: 14, Loss: -0.5486, Train: 49.45%, Valid: 53.89% Test: 56.76%
Epoch: 15, Loss: -0.5537, Train: 49.46%, Valid: 

In [ ]:
best_result = test(best_model, data, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 62.58%, Valid: 64.23% Test: 64.98%


#Conformal Prediction

since we didn't use labels of the nodes in validation set during training of the model and we didn't change the model based on validation set at all, we will use validation set as calibration set in this implementation

###initializing calibration set

In [ ]:
out = best_model(data.x, data.adj_t)
out.shape

torch.Size([169343, 40])

In [ ]:
calib_idx = split_idx['valid']
out[calib_idx].shape

torch.Size([29799, 40])

In [ ]:
print(data.y.min())
print(data.y.max())

tensor(0)
tensor(39)


In [ ]:
data.y[calib_idx]

tensor([[ 8],
        [19],
        [16],
        ...,
        [30],
        [16],
        [28]])

###Conformal Prediction Functions

In [ ]:
def get_DAPS_scores(adj_t, base_scores, lamb, iterations):
  num_nodes = adj_t.size(dim=0)
  degrees = adj_t.sum(dim=1)
  rows, cols, wieghts = adj_t.coo()

  A = torch.sparse_coo_tensor(
    torch.tensor([list(rows), list(cols)]),
    torch.ones(len(list(rows))),
    size=[num_nodes, num_nodes]
  )

  D_inv = torch.sparse_coo_tensor(
    torch.tensor([[_ for _ in range(num_nodes)], [_ for _ in range(num_nodes)]]),
    [1.0/d for d in degrees],
    size=[num_nodes, num_nodes]
  )

  cur_scores = base_scores.clone()
  for t in range(iterations):
    cur_scores = (1-lamb)*base_scores + lamb*D_inv@A@cur_scores

  return cur_scores

In [ ]:
DAPS_scores = get_DAPS_scores(data.adj_t, out, 0.5, 1)
DAPS_scores

tensor([[7.7288e-08, 6.9113e-08, 5.4967e-07,  ..., 1.4201e-07, 6.0433e-07,
         9.7248e-08],
        [1.7057e-09, 1.9465e-09, 8.7014e-07,  ..., 3.1826e-09, 3.8735e-09,
         2.2287e-09],
        [2.9829e-13, 4.3658e-13, 3.4426e-11,  ..., 1.3424e-12, 1.1977e-12,
         3.0237e-13],
        ...,
        [1.2087e-09, 2.5366e-10, 1.8979e-05,  ..., 2.4484e-09, 1.0351e-06,
         4.3633e-10],
        [8.9329e-11, 1.0907e-10, 9.6861e-10,  ..., 1.3626e-10, 1.0433e-09,
         1.5471e-10],
        [1.0031e-15, 1.0356e-15, 7.5853e-14,  ..., 1.4304e-14, 4.8314e-12,
         1.8413e-15]], grad_fn=<AddBackward0>)

In [ ]:
calib_scores = DAPS_scores[calib_idx, list(data.y[calib_idx])]
calib_scores

tensor([1.0000e+00, 5.1222e-13, 8.7491e-01,  ..., 9.3728e-01, 7.6589e-04,
        1.0000e+00], grad_fn=<IndexBackward0>)

In [ ]:
import math

alpha = 0.2
q_hat = torch.quantile(calib_scores, math.floor(alpha*(len(calib_idx) - 1))/len(calib_idx))
q_hat

tensor(5.1208e-07, grad_fn=<SqueezeBackward4>)

In [ ]:
(calib_scores >= q_hat).sum()/calib_scores.shape[0]

tensor(0.8000)

In [ ]:
conformal_labels = (DAPS_scores >= q_hat)

In [ ]:
print(f"the average of sizes of the sets is: {conformal_labels.sum(dim=1).float().mean()}")

the average of sizes of the sets is: 5.477498531341553


In [ ]:
node = 10
conformal_labels[node].nonzero()

tensor([[24],
        [28],
        [36]])